In [1]:
from PIL import Image, ImageFilter
import math
import time
from tqdm import tqdm

In [170]:
from pynput import mouse
top_left = None
def on_click(x, y, button, pressed):
    global top_left
    if not pressed:
        # Stop listener
        top_left = (x,y)
        return False
print("Click in the top left of Giga Manga Canvas")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print('Top Left',top_left)

bottom_right = None
def on_click(x, y, button, pressed):
    global bottom_right
    if not pressed:
        # Stop listener
        bottom_right = (x,y)
        return False
print("Click in the bottom right of Giga Manga Canvas")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print("Bottom Right", bottom_right)
s_width =  bottom_right[0]- top_left[0]
s_height = bottom_right[1]- top_left[1]
print('Selected Width', s_width)
print('Selected Height', s_height)

Click in the top left of Giga Manga Canvas
Top Left (1780, 324)
Click in the bottom right of Giga Manga Canvas
Bottom Right (2302, 789)
Selected Width 522
Selected Height 465


In [171]:
fc_image = Image.open("sailormoon.png")
image = fc_image.convert("L")
image = image.filter(ImageFilter.FIND_EDGES)
image.save(r"Edge_Sample.png")
print('image size:',(image.width,image.height))

image size: (344, 597)


In [172]:
w_padding = 0
h_padding = 0
if s_width > image.width:
    print('image width smaller than canvas, centering')
    w_padding = (s_width-image.width) /2
else:
    print("Your image is larger than your selected width")
if s_height > image.height:
    print('image height smaller than canvas, centering')
    h_padding = (s_height-image.height) /2
else:
    print("Your image is larger than your selected height")

print('w_padding',math.floor(w_padding))
print('h_padding',math.floor(h_padding))

image width smaller than canvas, centering
Your image is larger than your selected height
w_padding 89
h_padding 0


In [173]:
from pynput.mouse import Button, Controller
mouse_c = Controller() 

In [174]:
class Line():
    def __init__(self,xy1,xy2):
        self.xy1 = xy1
        self.xy2 = xy2
        
    def drag (self):
        mouse_c.position = self.xy1
        mouse_c.press(Button.left)
        for x in range(self.xy1[0],self.xy2[0]):
            for y in range(self.xy1[1],self.xy2[1]):
                mouse.position = (x,y)
        
        mouse_c.position = self.xy2
        mouse_c.release(Button.left)

In [175]:
class lineFinder():
    def __init__(self,image,resolution):
        self.image = image
        self.lines = []
        self.resolution =resolution
        
        
    def addLine(self,row):
        xy1 = min(row)
        xy2 = max(row)
        xy1 = ( int(xy1[0]+top_left[0]+w_padding) , int(xy1[1]+top_left[1]+h_padding))
        xy2 = ( int(xy2[0]+top_left[0]+w_padding) , int(xy2[1]+top_left[1]+h_padding))
        self.lines.append(Line(xy1,xy2))
        
    def detect(self):
        inRow=False
        image = self.image
        row = []
        for y in range(0,image.height,self.resolution):
            for x in range(0,image.width,self.resolution):
                pix = image.getpixel((x,y))
                if pix > 20:
                    if inRow:
                        row.append((x,y))
                    else:
                        inRow=True
                        row = []
                else:
                    if len(row)>1 and len(row) < image.width/2:
                        self.addLine(row)
                    inRow=False
                    row = []
            if len(row)>1 and len(row) < image.width/2:
                self.addLine(row)
            inRow=False
            row = []
                
                
        
        inRow=False
        for x in range(0,image.width,self.resolution):
            for y in range(0,image.height,self.resolution):
                pix = image.getpixel((x,y))
                if pix > 100:
                    if inRow:
                        row.append((x,y))
                    else:
                        inRow=True
                        row = []
                else:
                    if len(row)>1 and len(row) < image.height/2:
                        self.addLine(row)
                    inRow=False
                    row = []
            if len(row)>1 and len(row) < image.height/2:
                self.addLine(row)
            inRow=False
            row = []

In [176]:
a = lineFinder(image,1)

In [177]:
a.detect()
len(a.lines)

4202

In [178]:
for c in tqdm(a.lines):
    c.drag()
    time.sleep(0.1)

100%|██████████| 4202/4202 [07:23<00:00,  9.47it/s]


In [179]:
pallet_top_left = None
def on_click(x, y, button, pressed):
    global pallet_top_left 
    if not pressed:
        # Stop listener
        pallet_top_left = (x,y)
        return False
print("Click in the left topmost color on the pallet")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print("pallet_top_left", pallet_top_left)

Click in the left topmost color on the pallet
pallet_top_left (2381, 195)


In [181]:
pp_p = None
def on_click(x, y, button, pressed):
    global pp_p 
    if not pressed:
        # Stop listener
        pp_p = (x,y)
        return False
print("Click in the right bottom safe spot on the canvas")
with mouse.Listener(
        on_click=on_click) as listener:
    listener.join()
print("pp_p", pp_p)

Click in the right bottom safe spot on the canvas
pp_p (2299, 789)


In [182]:
def selectColorByRowAndCol(row,col):
    global mouse_c
    x = pallet_top_left[0] + ( col * 44)
    y = pallet_top_left[1] + (row * 44)
    mouse_c.position = (x,y) 
    mouse_c.press(Button.left)
    time.sleep(0.5)
    mouse_c.release(Button.left)
    
    time.sleep(1)

In [183]:
def convertIndextoRowAndColumn(index):
    row = int(index /7)
    col = index -(row*7)
    return (row,col)

In [184]:
from math import sqrt

COLORS = {
    0:[152,0,67], #magenta
    1:[122,1,120], #purple
    2:[36,50,147], # blue
    3:[6,94,53], #green
    4:[153,53,4], #brown
    5:[188,1,38], #red
    6:[34,34,34], #black
    
    7:[191,100,142], #pink
    8:[186,119,182], # light purple
    9:[120,138,191],# light blue
    10:[99,156,111],# light green
    11:[199,137,90],#light brown
    12:[222,126,120],# salmon
    13:[142,142,142], # gray
    
    14:[229,200,216], #lightest pink
    15:[249,236,243], #lightest purple
    16:[204,225,235], #lightest blue
    17:[191,217,168], #lightest green
    18:[245,221,176], #lightest brown
    19:[255,251,202], #lightest yellow
    20:[250,250,250], #white
    
}

def closest_color(rgb):
    r, g, b = rgb
    color_diffs = []
    for color in COLORS:
        cr, cg, cb = COLORS[color]
        color_diff = sqrt((r - cr)**2 + (g - cg)**2 + (b - cb)**2)
        color_diffs.append((color_diff, COLORS[color],color))
    return min(color_diffs)[2]

In [186]:
from collections import defaultdict
colorize = defaultdict(list)

for y in range(1,image.height,20):
    for x in range(1,fc_image.width,20):
        pix = fc_image.getpixel((x,y))[:3]
        
        pallet_color = closest_color(pix)
        px = int(x+top_left[0]+w_padding) 
        py=int(y+top_left[1]+h_padding)
        if py < pp_p[1]:
            colorize[pallet_color].append((px,py))
        

In [187]:
for item in colorize.keys():
    print(item,len(colorize[item]))

6 199
11 91
12 12
4 19
19 3
18 44
13 3
14 2
15 6
16 2
20 2
10 1
0 1
2 30
9 2
5 15


In [188]:
row,col = convertIndextoRowAndColumn(7)
selectColorByRowAndCol(row,col)
mouse_c.position = pp_p
#mouse_c.click(Button.left)

In [189]:
for color in colorize:
    row,col = convertIndextoRowAndColumn(color)
    selectColorByRowAndCol(row,col)
    mouse_c.position = pp_p
    mouse_c.click(Button.left)
    print(row,col)
    
    
    time.sleep(1)
    m = 10
    for xy in tqdm(colorize[color]):  
        mouse_c.position = xy
        mouse_c.press(Button.left)
        
        mouse_c.position = (xy[0]+1,xy[1])
        mouse_c.release(Button.left)
        

0 6


100%|██████████| 199/199 [00:00<00:00, 375.47it/s]


1 4


100%|██████████| 91/91 [00:00<00:00, 362.02it/s]


1 5


...

100%|██████████| 30/30 [00:00<00:00, 393.32it/s]


1 2


100%|██████████| 2/2 [00:00<00:00, 294.16it/s]


0 5


100%|██████████| 15/15 [00:00<00:00, 350.10it/s]
